In [1]:
import os, sys
from scipy import stats

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import tensorflow as tf
import numpy as np
from pprint import pprint
import keras
from keras import backend as K
from keras.models import Sequential, Model,load_model
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Input, Concatenate,concatenate, Reshape
from keras.wrappers.scikit_learn import KerasRegressor
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from keras.applications.densenet import DenseNet121
from sklearn.preprocessing import normalize

import pdb
import matplotlib.pyplot as plt

import collections
import numpy as np

import csv

%matplotlib inline

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17878011878058515067
]


Using TensorFlow backend.


In [2]:
baseDir = "/atlas/u/chenlin/wikipedia_images/Poverty_prediction/original_tfrecord/"
!ls "/atlas/u/chenlin/wikipedia_images/Poverty_prediction/original_tfrecord/"

lx_median_2009-11_angola_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_burkina_faso_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_cameroon_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_ethiopia_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_lesotho_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_malawi_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_mozambique_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_nigeria_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_rwanda_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_senegal_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_tanzania_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_uganda_dhslocs_ee_export.tfrecord
lx_median_2009-11_uganda_dhslocs_ee_export.tfrecord.gz
lx_median_2009-11_zimbabwe_dhslocs_ee_export.tfrecord.gz
lx_median_2012-14_benin_dhslocs_ee_export.tfrecord.gz
lx_median_2012-14_burkina_faso_dhslocs_ee_export.tfrecord.gz
lx_median_2012-14_cote_d_ivoire_dhslocs_ee_export.tfrecord.gz
lx_median_2012-14_drc_dhslocs_ee_ex

In [3]:
# ----------- Hyperparameters -----------

band_names = ['BLUE', 'GREEN', "RED"]#, "NIR", "SWIR1", "SWIR2"]
BATCH_SIZE = 32
EPOCH = 10 
TFRecord_files = [baseDir + 'lx_median_2015-17_nigeria_dhslocs_ee_export.tfrecord.gz']
TFRecord_test = [baseDir + 'lx_median_2015-17_kenya_dhslocs_ee_export.tfrecord.gz']

In [4]:
# ----------- Load Doc2Vec ------------
doc2vec_path = '/atlas/u/esheehan/wikipedia_project/dataset/GUF_dataset/data/AfricaArticleClustersUpdated.npy'
doc2vecs = np.load(doc2vec_path)

In [5]:
def load_model_cnn():
    np.random.seed(1234)
    model1 = Input(shape=(255, 255, 4))
    x = Conv2D(3,(1,1))(model1)
    base_model =  keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet', input_shape=(255, 255, 3))
    for layer in base_model.layers[:-3]:
        layer.trainable = False
    x = base_model(x)
    #base_model = load_model('/atlas/u/buzkent/Wikipedia_based_Learning/keras_model/wiki_global_full_FT.03.hdf5')
    x = Flatten()(x)
    x = Dense(3010)(x)
    model2 = Input(shape=(3010,))
    merge = concatenate([x, model2])
    merge = Dense(512, kernel_initializer='normal', activation='sigmoid')(merge)
    merge = Dense(128, kernel_initializer='normal', activation='sigmoid')(merge)
    merge = Dense(1, kernel_initializer='normal')(merge)
    model = Model(inputs=[model1, model2], outputs=merge)
    model.compile(loss='mse', optimizer='adam')#, metrics=[coeff_determination, 'mae'])
    print(model.summary())
    return model

In [6]:
def load_model_nightlight_summary():
    np.random.seed(1234)
    
    # For histogram
    model1 = Input(shape=(3010,1))
    x = Reshape((3010,), input_shape=(3010,1))(model1)
    
    # For summary stats
#     model1 = Input(shape=(514,1))
#     x = Reshape((514,), input_shape=(514,1))(model1)
#     x = Dense(3010, activation='sigmoid')(x)
    
    

    model2 = Input(shape=(3010,))
    merge = concatenate([x, model2])
    merge = Dense(512, kernel_initializer='normal', activation='sigmoid')(merge)
    merge = Dense(128, kernel_initializer='normal', activation='sigmoid')(merge)
    merge = Dense(1, kernel_initializer='normal')(merge)
    model = Model(inputs=[model1, model2], outputs=merge)
    model.compile(loss='mse', optimizer='adam')#, metrics=[coeff_determination, 'mae'])
    print(model.summary())
    return model

In [7]:
# Parses the TF record and returns everything
def _parse_function(example_proto):
    features = {
        band_name : tf.FixedLenFeature(shape=[255**2], dtype=tf.float32) for band_name in ['LAT','LON','RED','GREEN','BLUE','NIGHTLIGHTS']
    }
    features['wealthpooled'] = tf.FixedLenFeature(shape=[1], dtype=tf.float32)
    parsed_features = tf.parse_single_example(example_proto, features)
    for band_name in band_names:
        parsed_features[band_name] = tf.cast(parsed_features[band_name], tf.float64)
    parsed_features['wealthpooled'] = tf.cast(parsed_features['wealthpooled'], tf.float64)
    parsed_features['LAT'] = tf.cast(parsed_features['LAT'], tf.float64)
    parsed_features['LON'] = tf.cast(parsed_features['LON'], tf.float64)
    return parsed_features 

In [8]:
def find_embeding(doc2vecs, lat,lon):
    closest = float('inf')
    vec = None
    for i in range(len(doc2vecs)):
        if len(doc2vecs[i])>=5:
            temp = (lat-doc2vecs[i][0][0])**2 + (lon-doc2vecs[i][0][1])**2
            if temp<=closest:
                closest = temp
                vec = doc2vecs[i][5]
    if vec:
        return vec
    return np.zeros((1,300))

In [9]:
def createPlot(x, y, label, xName, yName):
    print("Plotting: ")
    fig, ax = plt.subplots()
    ax.plot(x, y, '-b', label=label)
    plt.xlabel(xName)
    plt.ylabel(yName)
    leg = ax.legend()
    plt.show()

def visualizeHistory(history):
    for mode in ["Train", "Val"]:
        for metric in ["loss", "mse", "r2", "spearmanRho", "spearmanP", "pearsonRho", "pearsonP"]:
            if mode == "Val" and metric == "loss":
                continue
            cur = []
            for epoch in range(EPOCH):
                cur.append(history[(epoch, mode + "_" + metric)])
                
            maxVal = max(cur)

            createPlot(range(EPOCH), cur, mode + " " + metric + " MaxVal: " 
                       +  str(maxVal), "Epoch", metric)


In [10]:
def find_embedding(doc2vecs, lat,lon):

    closest = float('inf')
    vec = None
    for i in range(len(doc2vecs)):
        if len(doc2vecs[i])>=5:
            temp = (lat-doc2vecs[i][0][0])**2 + (lon-doc2vecs[i][0][1])**2
            if temp<=closest:
                closest = temp
                vec = doc2vecs[i]
    if vec is not None:
        return concatenate_doc2vec(vec, 10), vec[2]
    print("None Found")
    return np.zeros((1,300)), None


In [11]:
def concatenate_doc2vec(i, num):
    embeddings = []
    for index, article in enumerate(i[6]):
        embeddings += list(article[4]) + [article[0]]
        if index == num - 1:
            break
    return embeddings

In [12]:
def runOneEpoch(epoch, datasetPath, sess, history, mode="Train"):
    dataset = tf.data.TFRecordDataset(datasetPath, compression_type="GZIP")
    image = dataset.map(_parse_function)  # Parse the record into tensors.
    image = image.batch(BATCH_SIZE)
    iterator_image = image.make_one_shot_iterator()
    next_image = iterator_image.get_next()
    
    all_labelsSave = []
    all_predsSave = []
    all_labels = []
    all_preds = []
    all_lat = []
    all_lon = []
        
    count = 0
    while True:
#         print('Now training batch {}, epoch {}'.format(count, epoch))
        Image = sess.run(next_image)
        Index = Image['wealthpooled']

# --------------- GET Images ---------------

#         r = Image['RED']
#         if r.shape[0] < BATCH_SIZE: 
#             break
#         r = np.array(r).reshape(BATCH_SIZE,255, 255)
#         g = Image['GREEN']
#         g = np.array(g).reshape(BATCH_SIZE,255, 255)
#         b = Image['BLUE']
#         b = np.array(b).reshape(BATCH_SIZE,255, 255)
#         n = Image['NIGHTLIGHTS']
#         n = np.array(n.reshape(BATCH_SIZE,255, 255))
#         stack_image = np.stack([r,g,b,n], axis=-1) 

        n = Image['NIGHTLIGHTS']
        if n.shape[0] < BATCH_SIZE: 
            break
        n = np.array(n.reshape(BATCH_SIZE,255, 255))
    
        # SUMMARY STATISTICS
#         summaryN = []
#         for curN in n: 
#             summaryN.append([np.max(curN), np.min(curN), np.mean(curN), 
#                              np.var(curN), *(stats.skew(curN)), *(stats.kurtosis(curN))])
#         summaryN = np.asarray(summaryN)
#         stack_image = np.stack([summaryN], axis=-1)
#         print(np.max(curN))
            
        # HISTOGRAM STATISTICS
        summaryN = []
        for curN in n: 
            tmp = curN
            tmp.flatten()
            hist = np.histogram(tmp, bins=np.linspace(0,255,3010), density = True)
            summaryN.append(np.asarray(hist[1]))

        summaryN = np.asarray(summaryN)
        stack_image = np.stack([summaryN], axis=-1)
        
        lat = np.mean(np.array(Image['LAT']),axis=1)
        lon = np.mean(np.array(Image['LON']),axis=1)

# --------------- GET DOC2Vec Embeddings --------------- 
        embedings = np.zeros((BATCH_SIZE,3010))
        for i in range(len(lat)):
            temp, index = find_embedding(doc2vecs, lat[i],lon[i])
            embedings[i,:] = temp
            if index is not None:
                Index[i] = index

# --------------- Metrics ---------------
        if mode == "Train":
            model_loss = model.train_on_batch([np.asarray(stack_image),embedings], np.concatenate(np.asarray(Index)))
        
        
        prediction = model.predict([np.asarray(stack_image),embedings], batch_size=BATCH_SIZE)

        all_labels.append(np.asarray(prediction))
        all_preds.append(np.asarray(Index))
            
        count += 1

        MSE = mean_squared_error(np.asarray(Index),np.asarray(prediction))
        if mode == "Train":
            history[(epoch, mode + '_loss')] += model_loss
        history[(epoch, mode + '_mse')] += MSE
    
    
# ------------- SAVING VALUES -------------
        for i in range(len(lat)):
            all_lat.append(lat[i])
            all_lon.append(lon[i])
            all_labelsSave.append(*prediction[i])
            all_predsSave.append(*Index[i])
        
    
# --------------- Summary Statistics ---------------
    if mode == "Train":
        history[(epoch, mode + '_loss')] /= count
    history[(epoch, mode + '_mse')] /= count
    
    r2Score =  r2_score((np.concatenate(all_labels, axis=0)), (np.concatenate(all_preds, axis=0)))
    (spearmanRho,spearmanP) = stats.spearmanr(np.concatenate(all_labels, axis=0), np.concatenate(all_preds, axis=0))
    (pearsonRho, pearsonP) = pearsonr(np.concatenate(all_labels, axis=0),np.concatenate(all_preds, axis=0))
    
    history[(epoch, mode + '_r2')] = r2Score
    history[(epoch, mode + "_spearmanRho")] = spearmanRho
    history[(epoch, mode + "_spearmanP")] = spearmanP
    history[(epoch, mode + "_pearsonRho")] = pearsonRho
    history[(epoch, mode + "_pearsonP")] = pearsonP
    
    print("____________________")
    print("Epoch {}, average {} loss: {}, ave mse: {}, ave R2: {}".format(epoch, mode,  history[(epoch, mode + '_loss')], 
                                                            history[(epoch, mode + '_mse')], history[(epoch, mode + '_r2')]))
    print("Spearman: {},{}  Pearson: {}, {} ".format(spearmanRho, spearmanP, pearsonRho, pearsonP))
    print("____________________")
    
    
    

# -------------- Return (lat,lon), predictions, results -------------
    return all_lat, all_lon, all_labelsSave, all_predsSave
    
    

In [13]:
def train(tfrecord_paths, tfrecord_test, model, filename = ""):
    
    history = collections.defaultdict(int)
    
    print("Train path:", tfrecord_paths)
    print("Test Path: ", tfrecord_test)
    image_size = 255
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(EPOCH):

            print("Starting epoch: {}".format(epoch))

            _, _, _, _ = runOneEpoch(epoch, tfrecord_paths, sess, history, mode="Train")
            _, _, _, _ = runOneEpoch(epoch, tfrecord_test, sess, history, mode="Val")
    
        print("Testing on filename: {}".format(tfrecord_test))
        all_lat, all_lon, all_labels, all_preds = runOneEpoch(epoch, tfrecord_test, sess, history, mode="Val")
        
        if filename != "":
            print("Saving to csv with filenames: {}, {}".format(filename + "_pred.csv", filename + "_true.csv" ))
            
            all_lat = ["lat"] + all_lat
            all_lon = ["lon"] + all_lon
            all_labels = ["wealthpooled"] + all_labels
            all_preds = ["wealthpooled"] + all_preds
            
            print(len(all_lat), len(all_lon), len(all_labels), len(all_preds))
            print(all_lat[0], all_lat[1])
            print(all_lon[0], all_lon[1])
            print(all_labels[0], all_labels[1])
            print(all_preds[0], all_preds[1])
            
#             np.savetxt(filename + '_pred.csv', [p for p in zip(all_lat, all_lon, 
#                                                 all_labels, all_preds)], delimiter=',', fmt='%s')
            
#             np.savetxt(filename + '_true.csv', [p for p in zip(all_lat, all_lon, 
#                                                 all_labels, all_preds)], delimiter=',', fmt='%s')

#             print("Saving history with filename {}".format(filename + ".npy"))
#             np.save(filename + ".npy", history)
        
    return history

In [14]:
rootPath = "/atlas/u/esheehan/wikipedia_project/dataset/GUF_dataset/output/doc2vecSat/"
filename = "nigeria-zimbabwe.npy"


# model = load_model_cnn()
model = load_model_nightlight_summary()
history = train(TFRecord_files, TFRecord_test, model, filename= rootPath + filename)  

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3010, 1)      0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 3010)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3010)         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 6020)         0           reshape_1[0][0]                  
                                                                 input_2[0][0]                    
__________

/sailhome/esheehan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/sailhome/esheehan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/sailhome/esheehan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/sailhome/esheehan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/sailhome/esheehan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


____________________
Epoch 7, average Val loss: 0, ave mse: 0.5633766620564824, ave R2: -1.0148891701227232e+16
Spearman: nan,nan  Pearson: [-4.79886984e-17], [1.] 
____________________
Starting epoch: 8


KeyboardInterrupt: 

In [ ]:
visualizeHistory(history)